In [1]:
cd /home/jovyan

/home/jovyan


In [2]:
!pip install pymongo --quiet

In [3]:
from lib.database import query_database, query_to_dataframe 

adult_for_modeling_df = query_to_dataframe("SELECT * FROM adult_for_modeling")

adult_for_modeling_df.shape

(30162, 13)

In [4]:
import pandas as pd

adult_for_modeling_encoded_df = pd.get_dummies(adult_for_modeling_df)

adult_for_modeling_encoded_df.shape

(30162, 102)

In [5]:
adult_for_modeling_encoded_df.columns

Index(['age', 'capital_gain', 'capital_loss', 'gender', 'hours_per_week',
       'income_label', 'education_ 10th', 'education_ 11th', 'education_ 12th',
       'education_ 1st-4th',
       ...
       'relationship_ Own-child', 'relationship_ Unmarried',
       'relationship_ Wife', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Private', 'workclass_ Self-emp-inc',
       'workclass_ Self-emp-not-inc', 'workclass_ State-gov',
       'workclass_ Without-pay'],
      dtype='object', length=102)

In [6]:
features = adult_for_modeling_encoded_df.drop("income_label", axis=1)
target = adult_for_modeling_encoded_df.income_label

In [7]:
from pymongo import MongoClient

In [8]:
mongo_client = MongoClient('this_mongo')
data_collection = mongo_client.data.datasets

In [9]:
import pickle

features_dict = {
    "dataset" : "features",
    "data" : pickle.dumps(features)
}

target_dict = {
    "dataset" : "target",
    "data" : pickle.dumps(target)
}

In [10]:
data_collection.insert_many([
    features_dict,
    target_dict
])

In [11]:
data_collection.count_documents({})

6

In [12]:
target_dict_from_mongo = data_collection.find_one({"dataset" : "target"})
target_from_mongo = pickle.loads(target_dict_from_mongo["data"])
features_dict_from_mongo = data_collection.find_one({"dataset" : "features"})
features_from_mongo = pickle.loads(features_dict_from_mongo["data"])